In [3]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/side_concordance_results.xlsx')


In [4]:
# def extract_side(location):
#     if pd.isnull(location):
#         return 'No Scan'
    
#     sides = []
    
#     if 'Left' in location:
#         sides.append('Left')
#     if 'Right' in location:
#         sides.append('Right')
    
#     if len(sides) == 0:
#         return 'Unknown'
#     elif len(sides) == 1:
#         return sides[0]
#     else:
#         return 'Both'


def extract_side(location):
    if pd.isnull(location):
        return 'No Scan'
    
    if 'Left' in location:
        return 'Left'
    elif 'Right' in location:
        return 'Right'
    else:
        return 'Unknown'

preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)

In [8]:
# Define modality
modality = '4D CT Scan'

# Apply extract_side function if not already done to get imaging sides
preop_imaging_df['4D CT Imaging Side'] = preop_imaging_df[modality].apply(extract_side)

# Filter to scans that localized anywhere (Left, Right)
localized_4dct = preop_imaging_df[preop_imaging_df['4D CT Imaging Side'].isin(['Left', 'Right'])]

# Count total localized scans
total_localized_4dct = localized_4dct.shape[0]

# Compare sides to see if there is at least a side match
# First extract surgical sides if not already done
preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)

# Filter to only rows in localized_4dct for side match calculation
localized_4dct = localized_4dct.copy()
localized_4dct['Side Match'] = localized_4dct['4D CT Imaging Side'] == localized_4dct['Surgical Side']

# Count how many matched on side (including exact matches)
side_match_count_4dct = localized_4dct['Side Match'].sum()



# Calculate percentage of scans that correlated with surgery (side match or exact match both count as a win here)
side_concordance_percentage_4dct = (side_match_count_4dct / total_localized_4dct) * 100 if total_localized_4dct > 0 else None

# Create a results dataframe for clean display
side_concordance_4dct_df = pd.DataFrame({
    'Metric': [
        'Total 4D CT scans with localization',
        'Scans matching surgical side or exact gland'
    ],
    'Value': [
        total_localized_4dct,
        f"{side_match_count_4dct} ({side_concordance_percentage_4dct:.2f}%)"
    ]
})

# Display the dataframe
display(side_concordance_4dct_df)





,Metric,Value
0,Total 4D CT scans with localization,171
1,Scans matching surgical side or exact gland,134 (78.36%)


In [9]:
# Define modality for SPECT/CT
modality_spect = 'SPECT/CT'

# Apply extract_side function if not already done to get imaging sides
preop_imaging_df['SPECT Imaging Side'] = preop_imaging_df[modality_spect].apply(extract_side)

# Filter to scans that localized anywhere (Left, Right, Both)
localized_spect = preop_imaging_df[preop_imaging_df['SPECT Imaging Side'].isin(['Left', 'Right', 'Both'])]

# Count total localized scans
total_localized_spect = localized_spect.shape[0]

# Extract surgical sides if not already done
preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)

# Filter to only rows in localized_spect for side match calculation
localized_spect = localized_spect.copy()
localized_spect['Side Match'] = localized_spect['SPECT Imaging Side'] == localized_spect['Surgical Side']

# Count how many matched on side (including exact matches)
side_match_count_spect = localized_spect['Side Match'].sum()

# Calculate percentage of scans that correlated with surgery
side_concordance_percentage_spect = (side_match_count_spect / total_localized_spect) * 100 if total_localized_spect > 0 else None

# Create a results dataframe for clean display
side_concordance_spect_df = pd.DataFrame({
    'Metric': [
        'Total SPECT/CT scans with localization',
        'Scans matching surgical side or exact gland'
    ],
    'Value': [
        total_localized_spect,
        f"{side_match_count_spect} ({side_concordance_percentage_spect:.2f}%)"
    ]
})

# Display the dataframe
display(side_concordance_spect_df)

,Metric,Value
0,Total SPECT/CT scans with localization,177
1,Scans matching surgical side or exact gland,151 (85.31%)


In [7]:
# Combine SPECT/CT and 4D CT side concordance dataframes with a Modality column
combined_side_concordance_df = pd.concat(
    [side_concordance_spect_df.assign(Modality='SPECT/CT'),
     side_concordance_4dct_df.assign(Modality='4D CT Scan')],
    ignore_index=True
)[['Modality', 'Metric', 'Value']]

# Display combined results
display(combined_side_concordance_df)

,Modality,Metric,Value
0,SPECT/CT,Total SPECT/CT scans with localization,177
1,SPECT/CT,Scans matching surgical side or exact gland,151 (85.31%)
2,4D CT Scan,Total 4D CT scans with localization,171
3,4D CT Scan,Scans matching surgical side or exact gland,134 (78.36%)
